 ╔══<i><b>Alai-DeepLearning</b></i>═══════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;week 4. Tensorflow **
# Section 5. Graph 관리하기 

### _Objective_
1. Tensor 또는 Node를 관리할 수 있는 방법에 대해 배워봅니다.  <br>
2. Graph와 Variable을 저장하고 불러오는 방법에 대해 배워봅니다.

╚════════════════════════════════════════╝

In [0]:
%matplotlib inline
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt

### [Optional.  Tensorflow Graph Visualization ]

---

> _Jupyter에서 Tensorflow에서 구성되는 Graph를 시각적으로 보여주기 위한 helper 메소드입니다._<br>


In [0]:
from IPython.display import clear_output, Image, display, HTML
import numpy as np    

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# \[ 1. 텐서 또는 노드를 관리하는 법 \]

---

---

> 텐서 또는 노드를 보관하고 관리하는 법을 배워 봅니다.

## 1.  Collections

---

+ Collections 은 텐서 또는 노드를 list 에 모아 보관할 수 있습니다. <br>


### (1) Collections 에 노드 또는 텐서를 추가하기

In [0]:
tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    a = tf.constant(3, name='a')
    b = tf.constant(3, name='b')
    tf.add_to_collection('consts_1', a)
    tf.add_to_collection('consts_1', b)

    c = tf.constant(4, name='c')
    d = tf.constant(5, name='d')
    tf.add_to_collections('consts_2',c)
    tf.add_to_collections('consts_2',d)    
    
    e = tf.add(a,b,name='Add')
    f = tf.multiply(c,d,name='Mul')
    tf.add_to_collection('result',e)
    tf.add_to_collection('result',f)

In [4]:
# 현재 그래프 내 모든 콜렉션의 키값을 가져옵니다.
graph.get_all_collection_keys()

['consts_1', 'consts_2', 'result']

### (2) Collections 에 노드 또는 텐서를 확인하는 법
콜렉션의 이름을 통해 원하는 텐서들만 가져올 수 있습니다.

In [5]:
my_col = graph.get_collection('consts_1')
my_col

[<tf.Tensor 'a:0' shape=() dtype=int32>,
 <tf.Tensor 'b:0' shape=() dtype=int32>]

### (3)  Variable의 Collection
변수 들이나 특정한 텐서 또는 노드들은 Tensorflow 가 자동으로 특정 Collections 에 추가합니다. Variables 은 자동으로 'variables' 라는 Collection 에 추가됩니다.  


In [6]:
tf.reset_default_graph()

graph = tf.Graph()
with graph.as_default():
    a = tf.Variable(3, name='a')
    b = tf.get_variable(initializer=3, name='b')

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
graph.get_all_collection_keys()

['trainable_variables', 'variables']

`Variable`는 자동으로 `variables`과 `trainable_variables`의 콜렉션에 들어갑니다.

In [11]:
import tensorflow as tf 
# traiable을 False하면 'variables'에만 변수가 들어갑니다.
# traiable을 False하면 'trainable_variables'에는 변수가 들어가지 않습니다. 
graph = tf.Graph()
with graph.as_default():
    b = tf.Variable(3, name='b',trainable=True)
    a = tf.Variable(3, name='a',trainable=False)
graph.get_all_collection_keys()
print(graph.get_collection('trainable_variables'))
print(graph.get_collection('variables'))

[<tf.Variable 'b:0' shape=() dtype=int32_ref>]
[<tf.Variable 'b:0' shape=() dtype=int32_ref>, <tf.Variable 'a:0' shape=() dtype=int32_ref>]


In [2]:
tf.get_collection('trainable_variables')

[]

### (4) 기본적으로 제공하는 Collection `tf.GraphKeys`

텐서플로우에서는 여러가지 주요한 컬렉션의 이름 집합을 제공합니다. 나중에 배울 High-API 텐서플로우 코드를 작성하다 보면, 자동으로 노드들이 의미에 맞게 컬렉션에 포함되게 됩니다.

|tf.GraphKeys | 의미 |
|----|---|
| INIT_OP | 초기화 관련된 노드 리스트들 |
| LOSSES  | 손실 함수에 관련된 노드 리스트들 |
| TRAIN_OP | optimizer 함수에 관련된 노드 리스트들 |
| TRAINABLE_VARIABLES | 학습되는 Variable 노드 리스트들 |
| SUMMARY_OP | 텐서보드에 넣기 위해 쓰이는 summary 관련 노드 리스트들 |

#### 예시 ) DNN 수업 때 다룰 코드 예시

텐서플로우의 코드를 군더더기 없이 짜기 위해서는, 의미에 맞게 `tf.GraphKeys`에 할당해주어야 합니다. 이러한 코드들은 나중에 다른 사람들이 읽었을 때에도 훨씬 더 가독성이 좋고, 유지보수 측면에서 매우 유리합니다.

In [0]:
graph = tf.Graph()
n_inputs = 784
n_hidden1 = 100
n_hidden2 = 100
n_hidden3 = 100
n_outputs = 10

with graph.as_default():
    X = tf.placeholder(tf.float32, (None,784), name='input')
    Y = tf.placeholder(tf.float32, (None,10), name='label')
    lr = tf.placeholder_with_default(0.01, None, name='learning_rate')
    init = tf.initializers.glorot_normal()
    
    # 1번째 ~ 3번째 은닉층
    n_nums = [n_hidden1, n_hidden2, n_hidden3]
    a = X
    for i, units in enumerate([n_hidden1, n_hidden2, n_hidden3]):
        with tf.variable_scope(f'hidden{i+1}'):
            a = tf.layers.dense(a, units, 
                                activation=tf.nn.relu,
                                kernel_initializer=init)
        
    logit = tf.layers.dense(a,n_outputs,kernel_initializer=init)

    loss = tf.losses.softmax_cross_entropy(Y,logit)
    
    with tf.variable_scope('metric'):
        labels = tf.argmax(Y, 1)
        correct = tf.nn.in_top_k(logit, labels, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) 
        
        tf.add_to_collection(tf.GraphKeys.METRIC_VARIABLES, accuracy)
    
    training_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
    
show_graph(graph)

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [0]:
# 학습해야 하는 Weight들을 가져오기 
graph.get_collection(tf.GraphKeys.VARIABLES)

Instructions for updating:
Use `tf.GraphKeys.GLOBAL_VARIABLES` instead.


[<tf.Variable 'hidden1/dense/kernel:0' shape=(784, 100) dtype=float32_ref>,
 <tf.Variable 'hidden1/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'hidden2/dense/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'hidden2/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'hidden3/dense/kernel:0' shape=(100, 100) dtype=float32_ref>,
 <tf.Variable 'hidden3/dense/bias:0' shape=(100,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(100, 10) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(10,) dtype=float32_ref>]

In [0]:
# Optimizer 함수 가져오기
graph.get_collection(tf.GraphKeys.TRAIN_OP)

[<tf.Operation 'GradientDescent' type=NoOp>]

In [0]:
# 손실 함수 가져오기 
graph.get_collection(tf.GraphKeys.LOSSES)

[<tf.Tensor 'softmax_cross_entropy_loss/value:0' shape=() dtype=float32>]

In [0]:
# 평가 지표 가져오기
graph.get_collection(tf.GraphKeys.METRIC_VARIABLES)

[<tf.Tensor 'metric/Mean:0' shape=() dtype=float32>]

### (5)  Collections 실행 시키기
Collection 은 list 로 되어 있습니다. Session 에서 Graph 을 실행시킬 때 list 을 넘겨주면 한번만 그래프를 실행 하고 list 에 담겨 있는 Tensor을  결과물로 반환합니다.

In [0]:
tf.reset_default_graph()
a = tf.constant(3, name='a')
b = tf.constant(5, name='b')
c = tf.constant(8, name='c')
tf.add_to_collection('my_collection', a)
tf.add_to_collection('my_collection', b)
tf.add_to_collection('my_collection', c)

my_col = tf.get_collection('my_collection')
sess = tf.Session()
sess.run(my_col)

[3, 5, 8]

### 예제 1)  
Collections 을 이용해 한번에  변수 initializer 실행 해보기 

변수 3개를 만들고 한번에 initializer 을 수행해 보세요.


In [0]:
tf.reset_default_graph()
a = tf.Variable(3, name='a')
b = tf.get_variable(initializer=6, name='b')
c = tf.Variable(initial_value=9, name='c')


tf.add_to_collection('initalizer' ,a.initializer)
tf.add_to_collection('initalizer' ,b.initializer)
tf.add_to_collection('initalizer' ,c.initializer)

init_col = tf.get_collection('initalizer')
vars_col = tf.get_collection('variables')
sess = tf.Session()
sess.run(init_col)
sess.run(vars_col)

[3, 6, 9]

## 2. Group

---

+ 노드 또는 텐서를 하나로 모아 실행 시킬수 있습니다. <br>


### (1) group 만들기 & 실행하기
tf.group 을 이용해 여러가지 텐서  또는 노드를 하나의 노드로 만들 수 있습니다. <br>
그리고 합쳐진 노드를 실행해 여러가지 텐서 또는 노드를 동시에 실행 합니다. 

In [0]:
tf.reset_default_graph()
a = tf.Variable(3, name='a')
b = tf.get_variable(initializer=3, name='b')

initializers = tf.group(a.initializer , b.initializer, 
                        name="Initializers")

In [0]:
default_graph = tf.get_default_graph()
show_graph(default_graph)

In [0]:
sess = tf.Session()
sess.run(initializers)

tf.group 의 반환값은 Node 입니다. Session 을 통해 Node 을 실행 시켜도 아무 값도 얻을수 없습니다.

### (2) global_variables_initializer  
변수를 만들때 마다 initializer 을 사용해 초기화 한다는 것은 너무 번거롭습니다.<br>
Tensorflow 에는 변수를 한번에 초기화 할 수 있는 `global_variables_initializer()` 함수를 제공 합니다 

In [0]:
tf.reset_default_graph()
a = tf.Variable(3, name='a')
b = tf.get_variable(initializer=3, name='b')
init = tf.global_variables_initializer()

In [0]:
default_graph = tf.get_default_graph()
show_graph(default_graph)

In [0]:
print(init)
sess = tf.Session()
sess.run(init)
sess.run(tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

name: "init"
op: "NoOp"
input: "^a/Assign"
input: "^b/Assign"



[3, 3]

### (3) train_op

딥러닝에서 parameter을 수정 하는 방법  중 하나는 Gradient Descent 방법 입니다.<br>
$\hat{w}= w - \frac{\delta Loss}{\delta w}  * learning\ rate  $<br>  
tf.train.GradientDescent 함수는 내부적으로 모든 trainable 변수에 위 공식을 적용하고 group 으로 묶습니다. 

In [0]:
tf.reset_default_graph()
x = 1.
y = 3.
w = tf.Variable(3., name='w')
lr = 1. 
pred = x * w 
loss = tf.square(y - pred)

w_grad = tf.gradients(ys=loss, xs=w)
w_hat =  w - w_grad[0] * lr 

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
print(sess.run([w_hat,w_grad]))

[3.0, [-0.0]]


In [0]:
tf.reset_default_graph()

x = tf.placeholder_with_default(input=1., shape=[], name='x')
y = tf.placeholder_with_default(input=1., shape=[], name='y')
w = tf.Variable(3., name='w')
lr = 1.
pred = x * w 

loss = tf.square(y - pred)

train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
sess.run(train_op)
print(sess.run(loss))

4.0


# \[ 2. Graph 및 변수를 저장 및 불러오기 \] 

---

---

> 텐서플로우에서는 Graph 및 변수를 저장 및 불러오는 방법을 제공합니다.

## 1. 그래프와 변수를 저장하기 

---

* Tensorflow 에서 정의된 Graph 와 변수를 저장 할 수 있습니다 

### (1) Graph 을 그린 후 변수의 값과 함께 저장합니다. 
tf.train.Saver() 을 이용해 Graph 와 변수 이름 , 그리고 변수 값 을 저장합니다. 

In [0]:
tf.reset_default_graph()
x = tf.constant(1.)
x1 = tf.constant(2.5)
y = tf.constant(4.)

w = tf.Variable(3., name='w')
w1 = tf.Variable(7.2, name='w')

lr = 1. 
pred = x * w + x1 * w1
loss = tf.square(y - pred)

w_grad = tf.gradients(ys=loss, xs=w)
w1_grad = tf.gradients(ys=loss, xs=w1)
w_hat =  w - w_grad[0] * lr 
w1_hat =  w1 - w1_grad[0] * lr 

w_assign = tf.assign(w, w_hat)
w1_assign = tf.assign(w, w_hat)

step = tf.group(w_assign , w1_assign)
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)
_, w_, w1_ = sess.run([step, w, w1])
print('w : {} w1 : {}'.format(w_, w1_))

# Graph 와  변경된 W, W1 값을 저장 합니다. 

saver = tf.train.Saver()
saver.save(sess, './tmp/model')

w : -31.0 w1 : 7.199999809265137


'./tmp/model'

In [0]:
!ls ./tmp

checkpoint                model.index
model.data-00000-of-00001 model.meta


위 코드를 실행하면 
아래 파일이 생성 되었음을 알 수 있습니다.
checkpoint <br>model.data-0000-of-00001 <br> model.index <br> model.meta


.data 파일은 각 변수의 값이 저장 되어 있습니다.  <br>
.index 파일에는 각 변수의 이름과 모양이 저장 되어 있습니다. <br>
.meta 파일에는 Graph 구조가 저장 되어 있습니다. <br>

### (2) 저장된 변수 목록 확인하기
`checkpoint_utils` 을 이용해 저장된 변수 이름과 모양을 확인 할 수 있습니다. 

In [0]:
from tensorflow.python.training import checkpoint_utils as cp

print('저장된 변수의 이름을 불러옵니다')
var_names = cp.list_variables('./tmp/model')
print('저장된 변수의 이름 {}'.format(var_names))

저장된 변수의 이름을 불러옵니다
저장된 변수의 이름 [('w', []), ('w_1', [])]


### (3) 저장된 변수 값 확인하기
`checkpoint_utils` 을 이용해 저장된 변수 목록과 값을 확인 할 수 있습니다. 

In [0]:
from tensorflow.python.training import checkpoint_utils as cp
w_value = cp.load_variable('./tmp/model' , 'w')
w1_value = cp.load_variable('./tmp/model' , 'w_1')

print('w : {} w1 : {}'.format(w_, w1_))

w : -31.0 w1 : 7.199999809265137


## 2. 그래프와 변수를 불러오기 
---

* 저장 된 Graph 와 변수를 불러 올수 있습니다. 


1. 우선 그래프를 그리거나 , 불러 와야 합니다. (graph restore)
2. 저장된 변수 값을 load 해야 합니다. (value restore)


### (1) Graph 을 그린 후 변수를 복제 합니다.
그래프를 똑같이 그린 후 변수를 그 그래프에 복제 합니다.

In [0]:
# Graph 을 그립니다. 
tf.reset_default_graph()
x = tf.constant(1.)
x1 = tf.constant(2.5)
y = tf.constant(4.)

w = tf.Variable(3., name='w')
w1 = tf.Variable(7.2, name='w')

saver = tf.train.Saver()
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# 변수를 초기화 합니다. 
print(sess.run(w))
print(sess.run(w1))

# 값을 불러와 해당 변수에 넣습니다.
saver.restore(sess, 'tmp/model')
print(sess.run(w))
print(sess.run(w1))

3.0
7.2
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from tmp/model
-31.0
7.2


### (2) Graph 불러온후 값 넣기
`.meta` 파일에 저장 되어 있는 그래프를 불러온 후 값을 채워 넣습니다. 


In [0]:
tf.reset_default_graph()
# meta 파일을 불러옵니다. 
tf.train.import_meta_graph('tmp/model.meta')
sess = tf.Session()
saver = tf.train.Saver()
saver.restore(sess, './tmp/model')
graph = tf.get_default_graph()
w = graph.get_tensor_by_name('w:0')
w1 = graph.get_tensor_by_name('w_1:0')
print(sess.run([w,w1]))

INFO:tensorflow:Restoring parameters from ./tmp/model
[-31.0, 7.2]


In [0]:
#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/03/22


---